텍스트 + 이미지 생성 모델
  - 1. 미리 훈련된 모델을 로드
    - DALL-E, DALL-E2는 openAI에서 제공하는 모델, 텍스트와 이미지를 결합해서 학습되었음
  - 2. 텍스트 프롬프트를 사용해서 이미지  생성
    - 사용자가 제공한 텍스트를 프롬프트 기반으로 모델은 텍스트의 의미를 이해하고 이를 시각적으로 표현하는 이미지를 생성

openai는 유료버전이므로 무료로 사용가능한 hugging face의 stable diffusion 모델을 사용 할 수 있음

In [1]:
!pip install diffusers transformers accelerate torch

In [2]:
# 허깅페이스 로그인
from huggingface_hub import notebook_login

notebook_login()

모델 다운로드 및 설정

In [1]:
import torch
from diffusers import StableDiffusionPipeline

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# stablediffusion 파이프 라인 로드
pipe = StableDiffusionPipeline.from_pretrained("sd-legacy/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe = pipe.to(device)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

이미지 생성 함수

In [2]:
def generate_image(prompt,output_path='generated_image.png'):
  '''
  텍스트 프롬프트 기반으로 이미지 생성(영어로)
  Args:
    prompt (str) : 텍스트 프롬프트
    output_path (str) : 생성된 이미지 저장 경로
  '''
  try:
    # 이미지 생성
    image = pipe(prompt).images[0]
    # 이미지 저장
    image.save(output_path)
  except Exception as e:
    print(f'이미지 생성 오류: {e}')


In [3]:
generate_image('a white cat')

  0%|          | 0/50 [00:00<?, ?it/s]

최적화
  - 해상도 증가
  - prompt 개선 **
  - scale 조정(Guidance Scale)  모델이 프롬프트에 얼마나 강하게 의존하는지 결정 7~12사이에 결정(기본 7.2)
  - 업스케일링(해상도 증가)
  - 모델 튜닝(Fine 튜닝)
  - 노이즈 조정(num_inference_steps) 값을 높이면 노이즈 제거 과정에서 더 많은 반복을 수행 70~100
  - VAE 교체 : 이미지의 색감과 디테일을 개선

In [4]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionLatentUpscalePipeline,StableDiffusionUpscalePipeline

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 업스케일러 로드
model_id = "stabilityai/stable-diffusion-x4-upscaler"
upscaler = StableDiffusionUpscalePipeline.from_pretrained(model_id, torch_dtype=torch.float16)
upscaler = upscaler.to(device)


model_index.json:   0%|          | 0.00/485 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

low_res_scheduler/scheduler_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/221M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/982 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
from PIL import Image
low_res_img = Image.open('generated_image.png').convert("RGB")
low_res_img = low_res_img.resize((512, 512))

# 업스케일링
upscaled_image = upscaler(prompt='a white cat',  image=low_res_img).images[0]
upscaled_image.save('upscaled_image_512.png')

  0%|          | 0/75 [00:00<?, ?it/s]